In [2]:
#import libraries
from datetime import datetime, timedelta,date
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division
from sklearn.cluster import KMeans


import plotly.graph_objs as go
import plotly.offline as pyoff

from sklearn.model_selection import KFold, cross_val_score, train_test_split


In [3]:
df_data = pd.read_csv('data/telco_churn.csv')

Encoding target variable

In [4]:
df_data.loc[df_data.Churn=='No','Churn'] = 0 
df_data.loc[df_data.Churn=='Yes','Churn'] = 1